## Non-Negative Matrix Factorization for Faster Recommendation

**Wikipedia:** Non-negative matrix factorization is a group of algorithms in multivariate analysis and linear algebra where a matrix V is factorized into (usually) two matrices W and H, with the property that all three matrices have no negative elements. This non-negativity makes the resulting matrices easier to inspect. Also, in applications such as processing of audio spectrograms or muscular activity, non-negativity is inherent to the data being considered. 

<img src="https://upload.wikimedia.org/wikipedia/commons/f/f9/NMF.png">

_Source: https://upload.wikimedia.org/wikipedia/commons/f/f9/NMF.png_

### Libraries

In [1]:
from surprise import SVD, SVDpp, NMF
from surprise import Dataset, accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import Reader
import pandas as pd
import time
import numpy as np

### Dataset Preparation

In [2]:
ratings = pd.read_csv('data/ratings.csv')

ratings_dict = {'itemID': ratings.movie_id_ml,
                'userID': ratings.user_id,
                'rating': ratings.rating
               }

df = pd.DataFrame(ratings_dict)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25)

genres= ['unknown','action','adventure','animation','childrens','comedy','crime','documentary','drama','fantasy','noir','horror','musical','mystery','romance','scifi','thriller','war','western']
column_item = ["movie_id_ml" ,"title",	"release", "vrelease", "url"] + genres

### Top-15 Most Voted Movies

In [3]:
mat = np.zeros((max(ratings.user_id), max(ratings.movie_id_ml)))
ind = np.array(list(zip(list(ratings.user_id-1), list(ratings.movie_id_ml-1))))
mat[ind[:,0], ind[:,1]] = 1
movies_ = mat.sum(axis=0).argsort()+1
np.random.shuffle(movies_)
top15 = movies_[:15]

In [4]:
pd.read_csv('data/u.item.txt', delimiter='|', encoding = "ISO-8859-1", header=None)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_ML_movies = pd.read_csv('data/u.item.txt', delimiter='|', names=column_item, encoding = "ISO-8859-1") 

In [6]:
min(ratings.user_id)

1

In [7]:
df_ML_movies.head()

,movie_id_ml,title,release,vrelease,url,unknown,action,adventure,animation,childrens,...,fantasy,noir,horror,musical,mystery,romance,scifi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:
df_ML_movies.movie_id_ml.isin(top15)

0       False
1       False
2       False
3       False
4       False
        ...  
1677    False
1678    False
1679    False
1680    False
1681    False
Name: movie_id_ml, Length: 1682, dtype: bool

In [9]:
top15

array([ 642,  306,  527,  475,  805,  755, 1022, 1556,  716,  547,  671,
       1519, 1313,  390, 1086])

In [10]:
df_ML_movies[df_ML_movies.movie_id_ml.isin(top15)]

,movie_id_ml,title,release,vrelease,url,unknown,action,adventure,animation,childrens,...,fantasy,noir,horror,musical,mystery,romance,scifi,thriller,war,western
305,306,"Mrs. Brown (Her Majesty, Mrs. Brown) (1997)",01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Her+Majesty%2...,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
389,390,Fear of a Black Hat (1993),01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?Fear%20of%20a...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
474,475,Trainspotting (1996),19-Jul-1996,NaN,http://us.imdb.com/Title?Trainspotting+(1996),0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
526,527,Gandhi (1982),01-Jan-1982,NaN,http://us.imdb.com/M/title-exact?Gandhi%20(1982),0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
546,547,"Young Poisoner's Handbook, The (1995)",23-Feb-1996,NaN,http://us.imdb.com/M/title-exact?Young%20Poiso...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
641,642,"Grifters, The (1990)",01-Jan-1990,NaN,"http://us.imdb.com/M/title-exact?Grifters,%20T...",0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
670,671,Bride of Frankenstein (1935),01-Jan-1935,NaN,http://us.imdb.com/M/title-exact?Bride%20of%20...,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
715,716,Home for the Holidays (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Home%20for%20...,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
754,755,Jumanji (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Jumanji%20(1995),0,1,1,0,1,...,1,0,0,0,0,0,1,0,0,0
804,805,Manhattan Murder Mystery (1993),01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?Manhattan%20M...,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [11]:
mat.sum(axis=0).argsort()

array([1681,  813, 1446, ...,   99,  257,   49])

In [12]:
mat.sum(axis=0)[0]

452.0

In [13]:
len(np.unique(ratings.user_id))

943

In [14]:
pd.read_csv('data/movies_cast_company.csv')

,movie_id_ml,title,release,url,unknown,action,adventure,animation,childrens,comedy,...,mystery,romance,scifi,thriller,war,western,movie_id,keyword,cast,company
0,1,toy story,1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,2445635,"['walkie-talkie', 'boy', 'slow-motion', 'villa...","[{""cast_id"": 193929, ""person_id"": 30260, ""cast...","[{""company_id"": 34, ""name"": ""Warner Home Video..."
1,2,goldeneye,1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,1,0,0,1923289,"['car-chase', 'good-versus-evil', '1990s', 'bl...","[{""cast_id"": 586283, ""person_id"": 83451, ""cast...","[{""company_id"": 19, ""name"": ""National Broadcas..."
2,3,four rooms,1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,1,0,0,1900170,"['number-in-title', 'title-directed-by-female'...","[{""cast_id"": 629008, ""person_id"": 89615, ""cast...","[{""company_id"": 11745, ""name"": ""Laurenfilm"", ""..."
3,4,get shorty,1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,1915485,"['actress', 'father-daughter-relationship', 'r...","[{""cast_id"": 1341029, ""person_id"": 184099, ""ca...","[{""company_id"": 19, ""name"": ""National Broadcas..."
4,5,copycat,1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,1,0,0,1788620,"['mother-son-relationship', 'san-francisco-cal...","[{""cast_id"": 643068, ""person_id"": 91412, ""cast...","[{""company_id"": 34, ""name"": ""Warner Home Video..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,1421,mi vida loca,1993,http://us.imdb.com/M/title-exact?Mi%20vida%20l...,0,0,0,0,0,0,...,0,0,0,0,0,0,2116298,"['title-directed-by-female', 'death-of-child',...","[{""cast_id"": 137914, ""person_id"": 22471, ""cast...","[{""company_id"": 240, ""name"": ""Home Box Office ..."
1311,1533,de eso no se habla,1993,http://us.imdb.com/M/title-exact?De%20Eso%20No...,0,0,0,0,0,0,...,0,0,0,0,0,0,1809271,"['title-directed-by-female', 'small-town', 'ar...","[{""cast_id"": 140778, ""person_id"": 22879, ""cast...","[{""company_id"": 521, ""name"": ""Argentina Video ..."
1312,1560,coup de torchon,1981,http://us.imdb.com/M/title-exact?Coup%20de%20t...,0,0,0,0,0,0,...,0,0,0,0,0,0,1790668,"['face-slap', 'based-on-novel', 'murder', 'bar...","[{""cast_id"": 801666, ""person_id"": 112621, ""cas...","[{""company_id"": 1705, ""name"": ""Criterion Colle..."
1313,1575,"yo, la peor de todas",1990,"http://us.imdb.com/M/title-exact?Yo,%20la%20Pe...",0,0,0,0,0,0,...,0,0,0,0,0,0,2513027,"['title-directed-by-female', 'marriage', 'moth...","[{""cast_id"": 214873, ""person_id"": 33360, ""cast...","[{""company_id"": 521, ""name"": ""Argentina Video ..."


### Training Matrix Factorization

In [15]:
start = time.time()
algo = NMF()
algo.fit(trainset)
predictions = algo.test(testset)
print("Test Set Error\n--------------")
accuracy.mae(predictions)
print("--------------\nFinished in {:.3f} sec.".format(time.time()-start))

# algo.pu -> User Matrix
# algo.qi -> Item Matrix

Test Set Error
--------------
MAE:  0.7627
--------------
Finished in 4.260 sec.


### Compare methods

In [16]:
import time
import datetime
import random

import numpy as np
import six
from tabulate import tabulate

from surprise.model_selection import KFold
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering

In [17]:
# The algorithms to cross-validate
classes = (SVD, SVDpp, NMF, SlopeOne, KNNBasic, KNNWithMeans, KNNBaseline,
           CoClustering, BaselineOnly, NormalPredictor)

# seeds
np.random.seed(0)
random.seed(0)

kf = KFold(random_state=0)  # folds will be the same for all algorithms.

In [18]:
table = []
for klass in classes:
    start = time.time()
    out = cross_validate(klass(), data, ['rmse', 'mae'], kf)
    cv_time = str(datetime.timedelta(seconds=int(time.time() - start)))

    mean_rmse = '{:.3f}'.format(np.mean(out['test_rmse']))
    mean_mae = '{:.3f}'.format(np.mean(out['test_mae']))

    new_line = [klass.__name__, mean_rmse, mean_mae, cv_time]
#     print(tabulate([new_line], tablefmt="pipe"))  # print current algo perf
    table.append(new_line)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity ma

In [19]:
header = ['RMSE', 'MAE', 'Time']
print(tabulate(table, header, tablefmt="pipe"))

|                 |   RMSE |   MAE | Time    |
|:----------------|-------:|------:|:--------|
| SVD             |  0.936 | 0.738 | 0:00:17 |
| SVDpp           |  0.922 | 0.723 | 0:10:56 |
| NMF             |  0.964 | 0.758 | 0:00:18 |
| SlopeOne        |  0.946 | 0.743 | 0:00:16 |
| KNNBasic        |  0.98  | 0.774 | 0:00:14 |
| KNNWithMeans    |  0.951 | 0.749 | 0:00:16 |
| KNNBaseline     |  0.931 | 0.733 | 0:00:17 |
| CoClustering    |  0.965 | 0.755 | 0:00:07 |
| BaselineOnly    |  0.944 | 0.748 | 0:00:02 |
| NormalPredictor |  1.523 | 1.222 | 0:00:01 |


### Try Collaborative Filtering with Singular Value Decomposition (SVD)

In [20]:
svd = SVD()

start = time.time()

cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
svd.fit(trainset)

predictions = svd.test(testset)

print("\nTest Set Error\n--------------")
accuracy.mae(predictions)
print("--------------\nFinished in {:.3f} sec.".format(time.time()-start))

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9361  0.9341  0.9393  0.9323  0.9321  0.9348  0.0027  
MAE (testset)     0.7360  0.7358  0.7401  0.7329  0.7351  0.7360  0.0023  
Fit time          2.91    2.88    2.90    2.87    2.88    2.89    0.01    
Test time         0.09    0.15    0.09    0.15    0.08    0.11    0.03    

Test Set Error
--------------
MAE:  0.7410
--------------
Finished in 18.539 sec.


In [21]:
ratings[ratings['user_id'] == 22]

,user_id,movie_id_ml,rating,rating_timestamp
2,22,377,1,1997-11-07 07:18:36
607,22,376,3,1997-11-07 07:18:32
680,22,128,5,1997-11-07 07:33:03
705,22,80,4,1997-11-07 07:20:27
1184,22,241,3,1997-11-07 07:33:45
...,...,...,...,...
95843,22,385,4,1997-11-07 07:31:09
96516,22,265,3,1997-11-07 07:34:26
96606,22,233,3,1997-11-07 07:34:26
98485,22,792,4,1997-11-07 07:10:47


In [22]:
# predict(user_id, movie_id) -> returns an estimated prediction of 2.21
svd.predict(22, 377, 3)

Prediction(uid=22, iid=377, r_ui=3, est=2.2469770791943535, details={'was_impossible': False})

### Using Non-Negative Matrix Factorization

In [23]:
algo = NMF()

start = time.time()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
algo.fit(trainset)

predictions = algo.test(testset)

print("\nTest Set Error\n--------------")
accuracy.mae(predictions)
print("--------------\nFinished in {:.3f} sec.".format(time.time()-start))

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9584  0.9638  0.9564  0.9572  0.9692  0.9610  0.0049  
MAE (testset)     0.7562  0.7578  0.7513  0.7538  0.7589  0.7556  0.0028  
Fit time          3.63    3.62    3.57    3.46    3.57    3.57    0.06    
Test time         0.14    0.12    0.07    0.12    0.07    0.11    0.03    

Test Set Error
--------------
MAE:  0.7616
--------------
Finished in 22.566 sec.
